In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py

In [3]:
from robust_motifs.timing import Timer
from robust_motifs.counting import get_n_extended_simplices
from robust_motifs.custom_mp import prepare_shared_memory
from pathlib import Path
import numpy as np
import scipy.sparse as sp
import pandas as pd

In [4]:
from robust_motifs.data import load_sparse_matrix_from_pkl, import_connectivity_matrix
from robust_motifs.plot import plot_matrices

In [5]:
from robust_motifs.simple_graphs import *
from pathlib import Path

In [6]:
import scipy.sparse as sp
import numpy as np
import h5py
import multiprocessing as mp
from itertools import product

In [7]:
funcs = [save_count_cyclic_extension, save_count_cyclic_extension_1_node, save_count_simplex_extension, save_count_circulant_extension_1_node]

In [8]:
n_nodes = [50, 50, 10, 10]

In [9]:
pool = mp.Pool()

In [10]:
for func, n in zip(funcs, n_nodes):
    path = Path("data/temp/test/simple/" + func.__name__)
    flag_path, matrix_path, count_path = func(path, n)
    
    matrix = load_sparse_matrix_from_pkl(matrix_path)
    matrix_info, links = prepare_shared_memory(matrix, 'f')
    bid_matrix = matrix.multiply(matrix.T)
    bid_matrix_info, bid_links = prepare_shared_memory(bid_matrix, 'c')
    
    c_file = h5py.File(count_path, 'r')
    
    print(func.__name__)
    
    for key in c_file.keys():
        simplices = c_file[key]
        mp_iterator = product(simplices, [matrix_info], [bid_matrix_info])
        
        count = 0
        
        r = pool.imap(get_n_extended_simplices, mp_iterator)
        for result in r:
            count += result[0]
            
        print(str(count) + " / " + str(len(simplices)))
        
    for link in links + bid_links:
        link.unlink()

/home/msantoro/miniconda3/envs/thesis38/lib/python3.8/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
100%|██████████| 100/100 [00:00<00:00, 561486.48it/s]
150it [00:00, 192517.01it/s]
100%|██████████| 51/51 [00:00<00:00, 141101.26it/s]
150it [00:00, 188029.17it/s]
100%|██████████| 20/20 [00:00<00:00, 92283.92it/s]
65it [00:00, 263665.15it/s]

save_count_cyclic_extension
50 / 150
save_count_cyclic_extension_1_node
2500 / 150
2400 / 150
save_count_simplex_extension


45 / 65
120 / 120
210 / 210
252 / 252


100%|██████████| 11/11 [00:00<00:00, 50589.19it/s]
40it [00:00, 215368.63it/s]

210 / 210
120 / 120
45 / 45
10 / 10
1 / 1
save_count_circulant_extension_1_node
110 / 40
170 / 70
70 / 40
